In [ ]:
import csv
import json
import os
import re
import time

import gym
import roboschool

import numpy as np

from gym import wrappers
from ipywidgets import Video
from multiprocessing import Pool


## Load a Model

Since we use a custom initializer and this gets serialized during the saving process of the model we need to pass it on when we load it again. Unfortunately with the issue of the background TensorFlow session when importing TensorFlow and multiprocessing we cannot define the initializer one time and use it here again. So we define it twice, one inside the create_model() function and here.

In [ ]:
def load_model(model_path):   
    import tensorflow as tf

    class Normc_initializer(tf.keras.initializers.Initializer):
        """
        Create a TensorFlow constant with random numbers normed in the given shape.
        :param std:
        :return:
        """
        def __init__(self, std=1.0):
            self.std = std

        def __call__(self, shape, dtype=None, partition_info=None):
            out = np.random.randn(*shape).astype(np.float32)
            out *= self.std / np.sqrt(np.square(out).sum(axis=0, keepdims=True))
            return tf.constant(out)
        
    class ObservationNormalizationLayer(tf.keras.layers.Layer):
        def __init__(self, ob_mean, ob_std, **kwargs):
            self.ob_mean = ob_mean
            self.ob_std = ob_std
            super(ObservationNormalizationLayer, self).__init__(**kwargs)

        def call(self, input):
            return tf.clip_by_value((input - self.ob_mean) / self.ob_std, -5.0, 5.0)
          
        def get_config(self):
            base_config = super(ObservationNormalizationLayer, self).get_config()
            base_config['ob_mean'] = self.ob_mean
            base_config['ob_std'] = self.ob_std
            return base_config
        
        @classmethod
        def from_config(cls, config):
            return cls(**config)
    
    custom_objects = {'Normc_initializer' : Normc_initializer(std=1.0), 
                      'ObservationNormalizationLayer' : ObservationNormalizationLayer}
    
    return tf.keras.models.load_model(model_path, custom_objects=custom_objects)

# Visualize

In [ ]:
def rollout(env, model, render=False, timestep_limit=None, random_stream=None):
    """
    If random_stream is provided, the rollout will take noisy actions with noise drawn from that stream.
    Otherwise, no action noise will be added.
    """

    env_timestep_limit = env.spec.tags.get('wrapper_config.TimeLimit.max_episode_steps')
    timestep_limit = env_timestep_limit if timestep_limit is None else min(timestep_limit, env_timestep_limit)
    rews = []
    t = 0

    ob = env.reset()
    for _ in range(timestep_limit):
        if render:
            env.render()
        ac = act(ob[None], model, random_stream=random_stream)[0]
        ob, rew, done, _ = env.step(ac)
        rews.append(rew)
        t += 1

        if done:
            break
    return np.array(rews, dtype=np.float32), t

def act(ob, model, random_stream=None):   
    action = model.predict(ob)
    
    #if random_stream is not None and model_structure.ac_noise_std != 0:
    #    action += random_stream.randn(*action.shape) * model_structure.ac_noise_std
    
    return action

In [ ]:
def index_save_directory(save_directory):
    if not os.path.isdir(save_directory):
        return None, None, None
    
    files = [save_directory + file for file in os.listdir(save_directory)]
    
    model_file_paths, log_file_path, config_file = [], None, None
    
    for file in os.listdir(save_directory):
        if file.endswith('.h5'):
            model_file_paths.append(file)
        elif file.endswith('config.json'):
            with open(save_directory + file, 'r') as f:
                config_file = json.load(f)
        elif file.endswith('log.txt'):
            log_file_path = save_directory + file
    
    model_file_paths.sort()
    
    return model_file_paths, log_file_path, config_file

def run_model(env_id, save_directory, model_file_path, record=False):
    env = gym.make(env_id)
    env.reset()
    
    if record:
        video_directory = save_directory + 'videos/' + model_file + '/'
        env = wrappers.Monitor(env, video_directory, force=True)
        
    model = load_model(save_directory + model_file_path)
    
    rewards, length = rollout(env, model)
    
    return np.array([rewards.sum(), length])

def visualize(save_directory):
    video_directories = []

    with Pool(os.cpu_count()) as pool:
        for m in model_files:
            video_directories.append(pool.apply_async(func=_visualize_in_process, args=(config_files['config']['env_id'], 
                                                        m,
                                                        save_directory,
                                                        True)))
        
        for i in range(len(video_directories)):
            video_directories[i] = video_directories[i].get()
        
    video_files = []
    for directory in video_directories:
        for file in os.listdir(directory):
            if file.endswith('.mp4'):
                video_files.append(directory + file)
    return video_files

In [ ]:
#video = Video.from_file(video_files[-1])
#video

In [ ]:
def parse_log_to_csv(log_file, csv_file):
    with open(log_file) as f:
        content = f.readlines()

    groups = temp =  []
    for line in content:
        line = line.split()

        if not line:
            continue

        if "Generation" in line:
            temp = [line[-1]]
            groups.append(temp)
        else:
            temp.append(line[-1])

    writer = csv.writer(open(csv_file, 'w'))

    writer.writerow(['Generation',
                     'Reward Mean',
                     'Reward Standard Deviation',
                     'Length Mean',
                     'Evaluation Reward Mean',
                     'Evaluation Reward Standard Deviation',
                     'Evaluation Length Mean',
                     'Evaluation Count',
                     'Episodes this generation',
                     'Episodes overall',
                     'Timesteps this generation',
                     'Timesteps overall',
                     'Unique Workers',
                     'Observation count',
                     'Time elapsed this generation (s)',
                     'Time elapsed overall (s)'])

    for generation in groups:
        if len(generation) != 18: continue

        row = []

        for column in generation:
            row.append(column)

        writer.writerow(row)


In [ ]:
def parse_generation_number(model_file_path):
    return int(re.split('/*/snapshot_|.h5', model_file_path)[0])

def evaluate(save_directory, eval_count=5, threshold_limit=None, record_all=False):
    if not save_directory.endswith('/'):
        save_directory += '/'
    
    model_file_paths, log_file_path, config_file = index_save_directory(save_directory)
    
    if model_file_paths is None or config_file is None:
        print("Not enough data to evaluate the training. Please provide a directory with enough data.")
        return
    
    
    if log_file_path is not None:    
        # Save parsed log to a csv spreadsheet
        csv_log_file_path = save_directory + 'log.csv'
        csv_log_file = parse_log_to_csv(log_file_path, csv_log_file_path)
    
    csv_eval_file_path = save_directory + 'evaluation.csv'
    
    writer = csv.writer(open(csv_eval_file_path, 'w'))
    
    head_row = ['Generation',
                'Evaluations per Generation'
                'Evaluation Reward Mean', 
                'Evaluation Reward Standard Deviation', 
                'Evaluation Length Mean']
    
    for i in range(eval_count):
        head_row.append('Reward ' + str(i))
        head_row.append('Lenght ' + str(i))
        
    writer.writerow(head_row)
        
    for model_file_path in model_file_paths:
        results = []
        with Pool(os.cpu_count()) as pool:
            for _ in range(eval_count):
                results.append(pool.apply_async(func=run_model, args=(config_file['config']['env_id'], 
                                                                      save_directory, 
                                                                      model_file_path)))
                
            for i in range(len(results)):
                results[i] = results[i].get()
        
        print(results)

        
        #rewards = np.array(results[:, 0])
        #lengths = np.array(results[:, 1]) TODO get this right -> get every value from first dimension
        
        row = [parse_generation_number(model_file_path),
               eval_count,
               np.mean(rewards),
               np.std(rewards),
               np.mean(lengths)]
        
        assert len(rewards) == len(lengths)
        for i in range(len(rewards)):
            row.append(rewards[i])
            row.append(lengths[i])
        
        writer.writerow(row)
    

In [ ]:
save_directory = '/tmp/es_4083/'

evaluate(save_directory)